# Challenge 14 "Unveiling the Secrets of Zurich’s Nightly Visitors (1780-1818)" /GLAMhack 2024

Entity extraction from OCR text

Author: Pablo Iriarte (UNIGE), pablo.iriarte@unige.ch

### API key
 
* API from  https://dandelion.eu: 2b089094a6e941429dc4472718eb604f

Example of request:

https://api.dandelion.eu/datatxt/nex/v1/?text=The%20doctor%20says%20an%20apple%20is%20better%20than%20an%20orange&include=types%2Cabstract%2Ccategories&token=<YOUR_TOKEN> 


``` json
{
  "timestamp": "Date and time of the response generation process",
  "time": "Time elapsed for generating the response (milliseconds)",
  "lang": "The language used to tag the input text",
  "langConfidence": "Accuracy of the language detection, from 0.0 to 1.0. Present only if auto-detection is on",
  "text": "The annotated text. Present only if the 'html' parameter has been used",
  "annotations": [
    {
      "id": "ID of the linked Wikipedia resource",
      "title": "Title of the linked Wikipedia resource",
      "uri": "URL of the entity on Wikipedia",
      "label": "Most common name used to represent the resource",
      "confidence": "Value of confidence for this annotation",
      "spot": "Annotated string, as it is in the input text",
      "start": "Character position in the input text where the annotation begins",
      "end": "Character position in the input text where the annotation ends",
      "types": ["List of types of the linked DBpedia resource","Only if 'include' parameter contains 'types'"],
      "categories": [
        "List of the category of the linked DBpedia resource",
        "Only if 'include' parameter contains 'categories'"
      ],
      "abstract": "Abstract of the linked Wikipedia resource. Only if 'include' parameter contains ­'abstract'",
      "lod": {
        "wikipedia": "URL of the Wikipedia article that represents the resource",
        "dbpedia": "URI of the resource on DBpedia"
      },
      "alternateLabels": [
        "List of other names used when referring to the entity",
        "Only if 'include' parameter contains 'alternate_labels'"
      ],
      "image": {
        "full": "URL of a depiction of the resource on Wikipedia. Only if 'include' parameter contains 'image'",
        "thumbnail": "URL of the thumbnail of the depiction. Only if 'include' parameter contains 'image'",
      }
    }
  ],
  "topEntities": [ # Only if 'top_entities' parameter is greater than 0
    {
      "id": "ID of the linked Wikipedia resource",
      "uri": "URL of the entity on Wikipedia",
      "score": "The result of the ranking algorithm"
    }
  ]
}

```
 

In [16]:
import sys
import os
import datetime
import shutil
import requests
from requests.exceptions import Timeout
import string
import io
import pandas as pd
import json
import codecs
import random
import time
import pandas as pd
from time import sleep
from datetime import date
from tqdm import tqdm

In [19]:
# loop to put text in dataframe
df = pd.DataFrame()
folders_txt = os.listdir('results')
for folder in folders_txt:
    texts = os.listdir('results/' + folder)
    for text in texts :
        if ('.txt' in text):
            print(text)
            with open('results/' + folder + '/' + text, 'r', encoding='utf-8') as file:
                data = file.read()
                # Name of the image file
                myimage = text.replace('.txt', '.jpg')
                df = pd.concat([df, pd.DataFrame.from_records([{'image' : myimage, 'text' : data, 'folder' : folder}])], ignore_index=True)

4648731.txt
4648733.txt
4648735.txt
4648737.txt
4648739.txt
4648741.txt
4648743.txt
4648745.txt
4648747.txt
4648749.txt
4648751.txt
4648753.txt
4648755.txt
4648757.txt
4648759.txt
4648761.txt
4648763.txt
4648765.txt
4648767.txt
4648769.txt
4648771.txt
4648773.txt
4648775.txt
4648777.txt
4648779.txt
4648781.txt
4648783.txt
4648785.txt
4648787.txt
4648789.txt
4648791.txt
4648793.txt
4648795.txt
4648797.txt
4648799.txt
4648801.txt
4648803.txt
4648805.txt
4648807.txt
4648809.txt
4648811.txt
4648813.txt
4648815.txt
4648817.txt
4648819.txt
4648821.txt
4648823.txt
4648825.txt
4648827.txt
4648829.txt
4648831.txt
4648833.txt
4648835.txt
4648837.txt
4648839.txt
4648841.txt
4648843.txt
4648845.txt
4648847.txt
4648849.txt
4648851.txt
4648853.txt
4648855.txt
4648857.txt
4648859.txt
4648861.txt
4648863.txt
4648865.txt
4648867.txt
4648869.txt
4648871.txt
4648873.txt
4648875.txt
4648877.txt
4648879.txt
4648881.txt
4648883.txt
4648885.txt
4648887.txt
4648889.txt
4648891.txt
4648893.txt
4648895.txt
4648

NotADirectoryError: [WinError 267] The directory name is invalid: 'results/export_txt_20240906.tsv'

In [20]:
df

,image,text,folder
0,4648731.jpg,"FREYTAGS, den 28 Julii 1780.\n\nSCHWERDT.\nHr....",1780-07
1,4648733.jpg,"NACHTZETTEL.\nSONNTAGS, den 30 Julii 1780.\n\n...",1780-07
2,4648735.jpg,"NACHTZETTEL\nMONTAGS, den 31 Julii 1780.\n\nSC...",1780-07
3,4648737.jpg,"NACHTZEDEL\nDIENSTAGS, den 1 Augusti 1780.\n\n...",1780-08
4,4648739.jpg,NACHTZEDEL.\nMITTWOCHS. den 2 Augusti 1780.\n\...,1780-08
...,...,...,...
1677,4665638.jpg,,1784-12
1678,4665639.jpg,nachtzebitt\n\n„„Mittwocht den Ei Dedempris.\n...,1784-12
1679,4665640.jpg,,1784-12
1680,4665641.jpg,H d £ a\n\nren\n\n_ Donftags den 48 Deeembris ...,1784-12


In [21]:
# skip rows in case of problem
myskip = -10

In [22]:
df.loc[df['text'] == '']

,image,text,folder
443,4652204.jpg,,1783-01
445,4652206.jpg,,1783-01
447,4652208.jpg,,1783-01
449,4652210.jpg,,1783-01
451,4652212.jpg,,1783-01
...,...,...,...
1673,4665634.jpg,,1784-12
1675,4665636.jpg,,1784-12
1677,4665638.jpg,,1784-12
1679,4665640.jpg,,1784-12


In [24]:
df.loc[df['text'] != '']

,image,text,folder
0,4648731.jpg,"FREYTAGS, den 28 Julii 1780.\n\nSCHWERDT.\nHr....",1780-07
1,4648733.jpg,"NACHTZETTEL.\nSONNTAGS, den 30 Julii 1780.\n\n...",1780-07
2,4648735.jpg,"NACHTZETTEL\nMONTAGS, den 31 Julii 1780.\n\nSC...",1780-07
3,4648737.jpg,"NACHTZEDEL\nDIENSTAGS, den 1 Augusti 1780.\n\n...",1780-08
4,4648739.jpg,NACHTZEDEL.\nMITTWOCHS. den 2 Augusti 1780.\n\...,1780-08
...,...,...,...
1672,4665633.jpg,"Sonntag8 den 26 Deczmbris 1784\na. SCHWEROT,\n...",1784-12
1674,4665635.jpg,"ee den 27 Desembfis un\nwe HWER DT,\nMi, Bay d...",1784-12
1676,4665637.jpg,NAchtzeben\n\nDienftags den 28 Decembris 1784;...,1784-12
1678,4665639.jpg,nachtzebitt\n\n„„Mittwocht den Ei Dedempris.\n...,1784-12


In [26]:
# loop to extract entities
total_errors = 0
headers = {'accept': 'application/json'}
# API key
api_key = '2b089094a6e941429dc4472718eb604f'
# ?text=The%20doctor%20says%20an%20apple%20is%20better%20than%20an%20orange&include=types%2Cabstract%2Ccategories&token= 
url_api = 'https://api.dandelion.eu/datatxt/nex/v1/'
mytimeout = 15
today = date.today()
date_today = today.strftime('%Y%m%d')
for index, row in df.iterrows():
    # skip some rows in case of 
    if (index >= 0):
        my_id = row['image']
        my_text = row['text']
        my_folder= row['folder']
        # test of text
        if (my_text != ''):
            print(my_id)
            try:
                r = requests.get(url_api + '?text=' + my_text + '&lang=de&include=types%2Ccategories%2Calternate_labels&token=' + api_key, headers=headers, timeout=mytimeout)
            except Timeout:
                print(my_id + ' | get error : TIMEOUT')
                total_errors = total_errors + 1
            else:
                # print(my_id + ' | item_get_status : ' + str(r.status_code))
                if (r.status_code == 200):
                    # save response
                    f = codecs.open('entities/' + my_id.replace('.jpg', '.json'), 'w', 'utf-8')
                    resp = r.text
                    resp = resp.replace('{"time"', '{"date_folder":"' + my_folder + '", "time"')
                    f.write(resp)
                    f.close()

4652711.jpg
4652713.jpg
4652715.jpg
4652717.jpg
4652718.jpg
4652719.jpg
4652721.jpg
4652723.jpg
4652725.jpg
4652727.jpg
4652729.jpg
4652731.jpg
4652733.jpg
4652735.jpg
4652737.jpg
4652739.jpg
4652741.jpg
4652743.jpg
4652745.jpg
4652747.jpg
4652749.jpg
4652751.jpg
4652753.jpg
4652755.jpg
4652757.jpg
4652759.jpg
4652761.jpg
4652763.jpg
4652765.jpg
4652767.jpg
4652769.jpg
4652771.jpg
4652773.jpg
4652775.jpg
4652777.jpg
4652779.jpg
4652781.jpg
4652783.jpg
4652785.jpg
4652786.jpg
4652787.jpg
4652789.jpg
4652791.jpg
4652793.jpg
4652795.jpg
4652797.jpg
4652798.jpg
4652799.jpg
4652801.jpg
4652803.jpg
4652805.jpg
4652807.jpg
4652809.jpg
4652813.jpg
4652815.jpg
4652817.jpg
4652819.jpg
4665023.jpg
4665025.jpg
4665027.jpg
4665029.jpg
4665031.jpg
4665033.jpg
4665035.jpg
4665037.jpg
4665039.jpg
4665041.jpg
4665043.jpg
4665045.jpg
4665047.jpg
4665049.jpg
4665051.jpg
4665053.jpg
4665055.jpg
4665057.jpg
4665059.jpg
4665061.jpg
4665063.jpg
4665065.jpg
4665067.jpg
4665069.jpg
4665071.jpg
4665073.jpg
4665